# Trabalho de Filtragem Colaborativa

Modelo de filtragem colaborativa usa as informações de ratings dos usuários para prover recomendações. <br>
**Ideia Principal:** A semelhança entre os usuários a partir do que se observa da interseção dos seus ratings permite inferir que os dados não inputados por apenas um destes poderia ter a mesma semelhança com relação ao outro. <br>
**Problema Principal:** A esparsidade dos dados, afinal a informação que um usuário provê é normalmente a um subconjunto muito pequeno dos itens. Logo a maioria da base de dados é de dados *faltantes* ou *não observados*. <br>
<br>
Há 2 métodos de filtragem colaborativa: <br>

**Memory-Based:** Também chamado de *neighborhood-based collaborative filtering algorithms*. Que se dividem basicamente em *user-based collaborative filtering* e *item-based collaborative filtering*. <br>
**Model-Based:** Modelos baseados em **machine learning** e **data mining** há um processo de aprendizado prévio para parametrizição. Alguns métodos são Decisions Trees, métodos bayesianos, modelos baseados em regras e latent factor method. <br>

## Implementação usando MovieLens
Carregar a base de 25M ou 100K


In [1]:
import pandas as pd         #DataFrames e operações associadas
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity          #Similaridade
import math  
import sklearn.metrics  

small = False  #usar a base de ratings reduzida ou não
ratings = pd.DataFrame()
movies = pd.DataFrame()

if (small):
    ratings = pd.read_csv('ratings_small.csv')
    movies = pd.read_csv('movies_small.csv').set_index("movieId")
else:
    ratings = pd.read_csv('ratings.csv')[:1000000] #ler as primeiras 2.000.000 linhas da base completa
    movies = pd.read_csv('movies.csv')
    movies = movies.set_index('movieId')

#### Funções para auxiliar
def listar_filmes_ja_vistos(usuario, matriz_filmes_X_usuarios):
    #filmes_ja_vistos_bin = matriz_filmes_X_usuarios.loc[usuario].gt(0)   #gerar array com o que usuário já deu rating: True ou False
    #return filmes_ja_vistos_bin.index[filmes_ja_vistos_bin].to_list() #com base no anterior, listar filmes que já viu        
    if(type(usuario)==list):
        filmes_usuario = matriz_filmes_X_usuarios.loc[usuario].sum(axis = 0)
    else: #if(type(usuario)==int):
        filmes_usuario = matriz_filmes_X_usuarios.loc[usuario]
    return filmes_usuario[filmes_usuario!=0].index.to_list()

def listar_filmes_avaliados(usuario_):
    if(type(usuario_)==list):
        return sorted(ratings[ratings['userId'].isin(usuario_)]['movieId'].unique().tolist())
    else:
        return ratings[ratings['userId']==usuario_]['movieId'].values.tolist()

def listar_filmes_nao_vistos(usuario, matriz_filmes_X_usuarios):
    if(type(usuario)==list):
        filmes_usuario = matriz_filmes_X_usuarios.loc[usuario].sum(axis = 0)
    else: 
        filmes_usuario = matriz_filmes_X_usuarios.loc[usuario]
    return filmes_usuario[filmes_usuario==0].index.to_list()

def eliminar_colunas_zeradas(matriz):
    return matriz.loc[:, (matriz != 0).any(axis=0)] #elimina todas as colunas cujos todos os valores são 0

def listar_nomes_filmes(indices):
    return movies.loc[indices]['title'].values.tolist()


In [2]:
n_ratings = len(ratings)
n_users = len(ratings['userId'].unique())
n_movies_avaliados = len(ratings['movieId'].unique())
print(f"Total de ratings: {n_ratings}")
print(f"Total de filmes: {len(movies)}")
print(f"Filmes avaliados: {n_movies_avaliados}")
print(f"Total de usuários: {n_users}")
print(f"Média de ratings/user: {round(n_ratings/n_users, 2)}")
print(f"Shape de Ratings: {ratings.shape}")
esparsidade = round(1.0 -n_ratings/float(n_users * n_movies_avaliados),3)
print(f"O nível de esparsidade do dataset é {esparsidade * 100}%")
ratings.sample(6).sort_index()


Total de ratings: 1000000
Total de filmes: 62423
Filmes avaliados: 21952
Total de usuários: 6747
Média de ratings/user: 148.21
Shape de Ratings: (1000000, 4)
O nível de esparsidade do dataset é 99.3%


,userId,movieId,rating,timestamp
8447,66,186,1.5,1138599882
14735,114,125914,2.5,1540562864
388975,2673,786,2.0,860750887
550909,3777,156726,3.5,1467168886
586220,4002,8937,3.5,1232156841
999435,6747,236,3.0,999377980


In [3]:
print("Relatório dos ratings dados pelos usuários:")
ratings.groupby('userId')['rating'].count().describe()

Relatório dos ratings dados pelos usuários:


count    6747.000000
mean      148.214021
std       237.566936
min        20.000000
25%        35.000000
50%        69.000000
75%       155.000000
max      4227.000000
Name: rating, dtype: float64

## Gerar matriz Movies X User

In [4]:
#movies_X_users = ratings.pivot_table(index="userId", columns="movieId", values="rating", fill_value=0)
# A LINHA DE CIMA FAZ O MESMO QUE A DE BAIXO GASTANDO 15X MAIS TEMPO
movies_X_users = ratings.groupby(['userId', 'movieId'])['rating'].first().unstack(fill_value=0.0)
movies_X_users

movieId,1,2,3,4,5,6,7,8,9,10,...,207367,207405,207642,207830,207890,208002,208080,208737,208793,208939
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6743,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6744,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6745,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
titulos_users = movies_X_users.copy() #se não mandar uma cópia, titulo users passa ser outro nome da mesma matriz e altera ela diretamente
titulos_users.columns = movies.loc[movies_X_users.columns.values.tolist()].title.values.tolist()
titulos_users.head(3)


,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Little Monsters (2019),Doctor Sleep (2019),Kabir Singh (2019),Terminator: Dark Fate (2019),Countdown (2019),The Kill Team (2019),Bruja (2019),Midway (2019),Watchman (2019),Klaus (2019)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## User-Based Collaborative Filtering

### Matriz de Similaridade por Usuário
A medida comumente usada é a similaridade do cosseno.
Essa medida de similaridade deve seu nome ao fato de ser igual ao cosseno do ângulo entre os dois vetores que estão sendo comparados:  vetores de similaridade de usuário (ou item) de ratings. Quanto menor o ângulo entre dois vetores, maior será o cosseno, resultando em um fator de similaridade mais alto. 

Dado 2 vetores, A e B, a similiridade por cosseno, cos($\theta$), é representada pelo produto escalar
$$\text{cosine similarity} =S_C (x,y):= \cos(\theta) = {\mathbf{x} \cdot \mathbf{y} \over \|\mathbf{x}\| \|\mathbf{y}\|} = \frac{ \sum\limits_{i=1}^{n}{x_i  y_i} }{ \sqrt{\sum\limits_{i=1}^{n}{x_i^2}}  \sqrt{\sum\limits_{i=1}^{n}{y_i^2}} }$$

In [6]:
users_cosine_array = cosine_similarity(movies_X_users)
users_cosine = pd.DataFrame(data=users_cosine_array, index=movies_X_users.index, columns=movies_X_users.index)
users_cosine.round(3).head()

userId,1,2,3,4,5,6,7,8,9,10,...,6738,6739,6740,6741,6742,6743,6744,6745,6746,6747
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000,0.041,0.061,0.041,0.016,0.000,0.094,0.021,0.023,0.026,...,0.028,0.018,0.046,0.023,0.125,0.043,0.000,0.022,0.047,0.025
2,0.041,1.000,0.179,0.197,0.158,0.130,0.065,0.177,0.129,0.157,...,0.083,0.256,0.148,0.110,0.113,0.082,0.263,0.136,0.223,0.250
3,0.061,0.179,1.000,0.358,0.061,0.115,0.031,0.081,0.062,0.132,...,0.017,0.192,0.087,0.016,0.216,0.202,0.140,0.089,0.201,0.117
4,0.041,0.197,0.358,1.000,0.066,0.072,0.016,0.089,0.066,0.084,...,0.013,0.092,0.096,0.011,0.122,0.168,0.106,0.055,0.184,0.095
5,0.016,0.158,0.061,0.066,1.000,0.115,0.202,0.308,0.216,0.270,...,0.180,0.077,0.268,0.166,0.107,0.059,0.156,0.148,0.163,0.261


### Selecionar a similaridade desejada

Nota: se fazer média das notas primeiro e usar cosseno, obtem-se mesmos valores que dá fazendo pearson diretamente. 
```python
    movies_X_users = movies_X_users - np.asarray([(np.mean(movies_X_users, 1))]).T
```
Implementação a seguir de **Pearson** não é recomendada pois gasta esta biblioteca é menos otimizada gastando cerca de 5x da implementação acima de cosseno 

$$r_{xy} =\frac{\sum ^n _{i=1}(x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum ^n _{i=1}(x_i - \bar{x})^2} \sqrt{\sum ^n _{i=1}(y_i - \bar{y})^2}}$$



In [7]:
#selecionar se vai usar pearson ou cosine para similaridade
usar_pearson = False
matriz_similaridade = users_cosine 
if (usar_pearson):     # Se verdadeiro, mudar para similaridade por pearson
    matriz_similaridade = movies_X_users.T.corr(method='pearson') #pode usar tb kendall e spearman

### Pegar os k usuários mais similares ao Target selecionado

In [8]:
def selecionar_usuarios_mais_similares(target = 1, k = 25):
    todas_similaridades_com_usuario = matriz_similaridade.loc[target].to_numpy()     #criar um array com a linha do target na matrix de similaridades
    usuarios_mais_similares = movies_X_users.index[todas_similaridades_com_usuario.argpartition(-k)[-k-1:-1]] #seleciona os k com similaridade mais alta no vetor excluindo ele mesmo
    return usuarios_mais_similares 

### Matriz de filmes não vistos pelo usuário target e que receberam notas dos usuários mais similares

In [9]:
target = 1
usuarios_mais_similares = selecionar_usuarios_mais_similares(1,)
#gerar matriz dos usuarios mais similares x filmes não assistidos ainda pelo usuário
usuarios_similares_X_filmes_nao_vistos = movies_X_users.loc[usuarios_mais_similares].drop(columns=listar_filmes_ja_vistos(1,movies_X_users)) 
usuarios_similares_X_filmes_nao_vistos = eliminar_colunas_zeradas(usuarios_similares_X_filmes_nao_vistos)
print('Matriz de filmes não vistos por usuários mais similares:',usuarios_similares_X_filmes_nao_vistos.shape)
usuarios_similares_X_filmes_nao_vistos.head(8)

Matriz de filmes não vistos por usuários mais similares: (25, 2965)


movieId,1,2,3,5,6,7,10,11,14,16,...,177593,178061,179085,180031,183837,183869,183897,188189,188697,201646
userId,,,,,,,,,,,,,,,,,,,,,
3469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
343,0.0,0.0,0.5,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
255,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2509,4.0,3.0,0.0,3.0,4.5,0.0,3.0,3.5,3.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5195,0.0,2.5,0.0,0.0,3.5,4.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4234,2.5,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Gerar a Recomendação de acordo com a nota dada pelos usuários similares

1. Para cada filme: 
2. Para cada usuário da lista de mais similar:
    1. Se nota foi dada: somar nota seguindo a fórmula
$$ nota_ = {\sum coeficienteUsuário * notaUsuário \over \sum coeficienteUsuário} $$

In [48]:
def predizer_notas(target, similares, filmes, min_threshold=0): # min_threshold: qtd mínima de notas para ser considerada ao recomendar um filme
    #Dataframe com a similaridade_dos_mais_similares: [userId, coeficiente de similaridade] 
    similaridade_dos_mais_similares = matriz_similaridade.loc[target,similares] 

    #matriz_similares_filmes = movies_X_users.loc[similares
    resultado = pd.DataFrame(columns=filmes, index=['Nota Final', '# Notas'] ) 
    resultado.columns.name = 'movieId'
    for filme in filmes:
        numerador = 0
        denominador = 0
        qtd_notas = 0
        for similar in similares:
            nota = movies_X_users.loc[similar,filme]
            if (nota != 0):
                coeficiente = similaridade_dos_mais_similares[similar]
                numerador += nota * coeficiente
                denominador += coeficiente
                qtd_notas += 1
        if (qtd_notas < min_threshold):
            resultado = resultado.drop([filme], axis=1) #se a qt de notas for menor que limiar, descartar coluna com informação daquele filme
        else: #se não, prencher a nota calculada da média ponderada e a qtd de notas dadas
            try:
                resultado.at['Nota Final',filme] = round(numerador/denominador,1)
            except:
                #print(f"filme:{filme}, numerador:{numerador}, denominador: {denominador}, qt. notas: {qtd_notas}.")
                resultado.at['Nota Final',filme] = 0
            resultado.at['# Notas',filme] = qtd_notas
    return resultado

In [11]:
qtd_sugestoes = 20 #qtd de sugestões para exibir na tela

filmes_vistos_pelo_usuario = listar_filmes_ja_vistos(target, movies_X_users) 
filmes_vistos_pelos_similares = listar_filmes_ja_vistos(usuarios_mais_similares.values.tolist(), movies_X_users)
filmes_a_avaliar = list(set(filmes_vistos_pelos_similares)-set(filmes_vistos_pelo_usuario))
recomendacao = predizer_notas(target, usuarios_mais_similares, filmes_a_avaliar, 5)
#recomendacao
recomendacao.T.sort_values(by=['Nota Final','# Notas',],ascending=False).head(qtd_sugestoes).join(movies[['title']], on=["movieId"])


,Nota Final,# Notas,title
movieId,,,
1464,4.8,6,Lost Highway (1997)
6985,4.7,6,"Passion of Joan of Arc, The (Passion de Jeanne..."
1211,4.6,7,"Wings of Desire (Himmel über Berlin, Der) (1987)"
668,4.6,5,Song of the Little Road (Pather Panchali) (1955)
8125,4.6,5,Sunrise: A Song of Two Humans (1927)
858,4.5,15,"Godfather, The (1972)"
924,4.5,13,2001: A Space Odyssey (1968)
3910,4.5,9,Dancer in the Dark (2000)
5971,4.5,8,My Neighbor Totoro (Tonari no Totoro) (1988)


## Avaliando a eficácia do método
1. Selecionar um target randômico
2. Selecionar alguns valores de notas dadas por ele
3. Tentar predizer sua nota com base nos seus similares, 
4. Calcular margem de erro 

Uma medida frequentemente usada na verificação da acurácia de modelos numéricos é o Erro Quadrático Médio (MSE na sigla em Inglês) como descrito, por exemplo, em Wilks (2006).MSE é sempre positivo. MSE = 0 indica simulação perfeita. MSE é definido por:
$$ MSE = \frac{1}{n} \Sigma_{i=1}^n({y}-\hat{y})^2 $$

Em adição ao MSE, a raiz quadrada de MSE, ou Raiz do Erro Quadrático Médio (RMSE em Inglês), é comumente usada para expressar a acurácia dos resultados numéricos com a vantagem de que RMSE apresenta valores do erro nas mesmas dimensões da variável analisada. O RMSE é definido por:
$$ RMSE = \sqrt{\frac{1}{n} \Sigma_{i=1}^n({y}-\hat{y})^2} $$

In [54]:
rnd_user = movies_X_users.sample()
rnd_user = eliminar_colunas_zeradas(rnd_user)
rnd_user

movieId,2,3,6,10,16,19,20,21,25,32,...,91529,91542,91983,92768,93510,94677,94777,94864,95032,95720
userId,,,,,,,,,,,,,,,,,,,,,
3046,3.0,2.5,4.5,3.0,4.5,3.0,3.0,3.0,3.0,3.0,...,4.0,3.0,3.0,3.5,3.0,2.5,3.0,3.0,3.0,2.5


In [55]:
rnd_user_id = rnd_user.index.values[0]

filmes_assistidos = listar_filmes_ja_vistos(rnd_user_id, movies_X_users)
similares = selecionar_usuarios_mais_similares(rnd_user_id)
predicao = predizer_notas(rnd_user_id, similares, filmes_assistidos, 0)
predicao


movieId,2,3,6,10,16,19,20,21,25,32,...,91529,91542,91983,92768,93510,94677,94777,94864,95032,95720
Nota Final,2.8,3.0,3.8,3.2,3.9,2.6,2.8,3.1,3.7,3.8,...,3.6,3.2,3.0,3.5,3.3,2.5,3.2,3.2,3.0,2.3
# Notas,18,9,22,22,22,18,3,14,10,25,...,7,5,1,1,5,3,4,4,1,2


In [56]:
def calcular_rmse(real, previsao):
    mse = sklearn.metrics.mean_squared_error(notasReais, notasPreditas)     
    #mse = np.square(np.subtract(notasReais,notasPreditas)).mean()      
    print("Erro Quadrático Médio (MSE):", mse)         
    return math.sqrt(mse)  

notasReais = rnd_user.values.tolist()[0]
notasPreditas = predicao.loc["Nota Final"].values.tolist()
print("Raiz do Erro Quadrático Médio :", calcular_rmse(notasReais, notasPreditas))  

Erro Quadrático Médio (MSE): 0.2856267029972752
Raiz do Erro Quadrático Médio : 0.5344405514154733


---
---

## Item-Based Collaborative Filtering

### Gerar matriz Users X Movies
Transposição da matriz que tinha usuários nas linhas e filmes nas colunas, para filmes nas linhas e usuários nas colunas

In [57]:
users_X_movies = movies_X_users.T
users_X_movies.head(4)

userId,1,2,3,4,5,6,7,8,9,10,...,6738,6739,6740,6741,6742,6743,6744,6745,6746,6747
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,4.0,3.0,4.0,0.0,0.0,4.0,0.0,3.5,...,0.0,0.0,3.0,0.0,0.0,4.0,4.0,0.0,0.0,5.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


### Matriz de Similaridade Item a Item
Similaridade por Cosseno dos filmes entre si

In [58]:
movies_cosine_array = cosine_similarity(users_X_movies)
movies_cosine = pd.DataFrame(data=movies_cosine_array, index=users_X_movies.index, columns=users_X_movies.index)
movies_cosine.head()
#movies_pearson = movies_users.corr(method='pearson')

movieId,1,2,3,4,5,6,7,8,9,10,...,207367,207405,207642,207830,207890,208002,208080,208737,208793,208939
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.382684,0.284309,0.111599,0.290132,0.357890,0.297022,0.101180,0.163940,0.358125,...,0.009647,0.008441,0.0,0.012354,0.023163,0.0,0.015442,0.0,0.025737,0.000000
2,0.382684,1.000000,0.227409,0.149937,0.232203,0.265937,0.204352,0.170280,0.166641,0.404381,...,0.023281,0.019643,0.0,0.000000,0.000000,0.0,0.037267,0.0,0.037267,0.023292
3,0.284309,0.227409,1.000000,0.195838,0.396649,0.281498,0.356806,0.102130,0.234407,0.224702,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
4,0.111599,0.149937,0.195838,1.000000,0.169821,0.102581,0.154230,0.150039,0.145440,0.121652,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
5,0.290132,0.232203,0.396649,0.169821,1.000000,0.238935,0.372944,0.133212,0.191455,0.179568,...,0.025562,0.000000,0.0,0.000000,0.000000,0.0,0.040919,0.0,0.000000,0.000000


### Selecionar um usuário target e seus filmes favoritos
É obtido a nota mais alta que este usuário deu e todos os filmes  com a mesma nota. Estes chamaremos de filmes favorito e de acordo eles é buscada a similaridade entre os itens

<center><img src="img/item-based-cosseno-predicao.jpg"></center>

In [63]:
usuarioId_target = 1
filmes_avaliados = listar_filmes_avaliados(usuarioId_target)
len(filmes_avaliados)

70

In [ ]:
usuarioId_target = 1
usuario = movies_X_users.loc[usuarioId_target].sort_values(ascending=False)
rate_mais_alto = usuario.iloc[0]
filmes_mais_gosta = usuario[usuario >= rate_mais_alto].index.tolist()
assistiu_n_filmes = len(usuario[usuario > 0].index.to_list())
print(f"O usuário {usuarioId_target} assistiu {assistiu_n_filmes} filmes e deu nota {rate_mais_alto} para estes {len(filmes_mais_gosta)} filmes: ")
print(listar_nomes_filmes(filmes_mais_gosta, movies))

O usuário 1 assistiu 70 filmes e deu nota 5.0 para estes 17 filmes: 
['Night, The (Notte, La) (1960)', 'Saragossa Manuscript, The (Rekopis znaleziony w Saragossie) (1965)', 'Seventh Seal, The (Sjunde inseglet, Det) (1957)', 'Run Lola Run (Lola rennt) (1998)', 'Dolls (2002)', 'Three Colors: Blue (Trois couleurs: Bleu) (1993)', 'Eternal Sunshine of the Spotless Mind (2004)', 'Pulp Fiction (1994)', 'Dolce Vita, La (1960)', 'Requiem for a Dream (2000)', 'Underground (1995)', 'City of God (Cidade de Deus) (2002)', 'Lost in Translation (2003)', 'Idiots, The (Idioterne) (1998)', 'In the Mood For Love (Fa yeung nin wa) (2000)', 'Teddy Bear (Mis) (1981)', 'Look at Me (Comme une image) (2004)']


### Similaridade dos filmes favoritos com os filmes não assistidos
Matriz onde cada um dos filmes favoritos é um índice e cada coluna é um filme que não foram assistidos ainda.

In [ ]:
lista_filmes_ja_vistos = listar_filmes_ja_vistos(usuarioId_target,movies_X_users)
print(f"Usuario {usuarioId_target} já assistiu {len(lista_filmes_ja_vistos)} filmes.")
filmes_pro_usuario = movies_cosine.loc[filmes_mais_gosta].drop(columns=lista_filmes_ja_vistos)
filmes_pro_usuario = eliminar_colunas_zeradas(filmes_pro_usuario) #eliminar as colunas dos filmes que não similaridade nenhuma com nada
filmes_pro_usuario

Usuario 1 já assistiu 70 filmes.


movieId,1,2,3,4,5,6,7,8,9,10,...,208531,208545,208683,208715,208737,208787,208793,208795,208939,209163
movieId,,,,,,,,,,,,,,,,,,,,,
4325,0.048319,0.036425,0.004197,0.000000,0.003165,0.052934,0.010252,0.000000,0.005654,0.030384,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.212605
2632,0.033449,0.029535,0.014118,0.000000,0.002129,0.040721,0.009906,0.000000,0.000000,0.026479,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1237,0.145767,0.101500,0.049393,0.038993,0.055269,0.148725,0.056772,0.033396,0.018669,0.105733,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058498,0.000000,0.040948
2692,0.280149,0.182277,0.097522,0.050761,0.071719,0.247829,0.088598,0.025237,0.039103,0.194257,...,0.000000,0.022133,0.025295,0.000000,0.000000,0.012647,0.031619,0.031619,0.022133,0.022133
8327,0.039146,0.040715,0.005662,0.000000,0.009340,0.045019,0.013325,0.000000,0.000000,0.023850,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.179259
307,0.137478,0.087935,0.057192,0.056846,0.051696,0.163889,0.085958,0.021172,0.035518,0.112383,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.056588,0.000000,0.039612
7361,0.343043,0.227357,0.074001,0.025174,0.082209,0.220624,0.072910,0.017835,0.028312,0.175972,...,0.018354,0.002294,0.018354,0.020648,0.021875,0.000000,0.000000,0.022943,0.018354,0.016060
296,0.491905,0.342615,0.164180,0.099265,0.151422,0.411448,0.158854,0.046971,0.100742,0.423525,...,0.014375,0.014375,0.014375,0.011500,0.012884,0.005750,0.014375,0.014375,0.014375,0.011500
8154,0.083079,0.061808,0.024624,0.021910,0.026139,0.091842,0.056287,0.010164,0.014778,0.058553,...,0.100019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Transformamos a matriz numa 1xN filmes com o valor máximo encontra de similaridade; e ordenamos essa matriz, selecionando os k filmes com maior similaridade apresentada <br>
Feito também um **join** com movies para mostrar o título

In [ ]:
k = 20
#pegar a similaridade máxima que cada um dos filmes não vistos possui com os filmes já vistos
recomendacao = filmes_pro_usuario.max().sort_values(ascending=False).head(k)
recomendacao = pd.DataFrame(recomendacao).join(movies['title'], on='movieId')
recomendacao

,0,title
movieId,,
593,0.698877,"Silence of the Lambs, The (1991)"
26094,0.687403,"Eclisse, L' (Eclipse) (1962)"
308,0.685197,Three Colors: White (Trzy kolory: Bialy) (1994)
318,0.665221,"Shawshank Redemption, The (1994)"
47,0.650256,Seven (a.k.a. Se7en) (1995)
50,0.646176,"Usual Suspects, The (1995)"
356,0.642366,Forrest Gump (1994)
2959,0.600179,Fight Club (1999)
1089,0.591013,Reservoir Dogs (1992)


In [ ]:
recomendacao = recomendacao.rename(columns={'title': 'Recomendação', 0: 'Score'})
pq_vc_assistiu = []
nota_media = []
for id in recomendacao.index:
    pq_vc_assistiu.append(filmes_pro_usuario.index[filmes_pro_usuario[id] == recomendacao.loc[id][0]].tolist()[0])
    nota_media.append(ratings.groupby(['movieId'])['rating'].mean().loc[id])
recomendacao['Nota média'] = nota_media
recomendacao['Pq vc assistiu'] = listar_nomes_filmes(pq_vc_assistiu)
recomendacao

,Score,Recomendação,Nota média,Pq vc assistiu
movieId,,,,
593,0.698877,"Silence of the Lambs, The (1991)",4.153263,Pulp Fiction (1994)
26094,0.687403,"Eclisse, L' (Eclipse) (1962)",3.800000,"Night, The (Notte, La) (1960)"
308,0.685197,Three Colors: White (Trzy kolory: Bialy) (1994),3.932000,Three Colors: Blue (Trois couleurs: Bleu) (1993)
318,0.665221,"Shawshank Redemption, The (1994)",4.429786,Pulp Fiction (1994)
47,0.650256,Seven (a.k.a. Se7en) (1995),4.090097,Pulp Fiction (1994)
50,0.646176,"Usual Suspects, The (1995)",4.307735,Pulp Fiction (1994)
356,0.642366,Forrest Gump (1994),4.045913,Pulp Fiction (1994)
2959,0.600179,Fight Club (1999),4.219444,Pulp Fiction (1994)
1089,0.591013,Reservoir Dogs (1992),4.101577,Pulp Fiction (1994)


---
---

# SVD: Fatoração de Matriz
Devido a esparsidade do dataset, os métodos tradicionais de filtragem colaborativa podem não serem adequados a demanda de processamento. Uma forma de tratar é fazendo uso do algoritmo de **Singular Value Decomposition**, SVD.<br>
Neste algoritmo, a matriz é decomposta em  em outras 3 matrizes de menor dimensionalidade.
$$ A = USV^T$$
- A é a matriz original m x n
- U é uma matriz ortogonal m x n
- S é uma matriz diagona n x n
- V é uma matriz ortogonal n x n

https://heartbeat.comet.ml/recommender-systems-with-python-part-iii-collaborative-filtering-singular-value-decomposition-5b5dcb3f242b

https://www.kaggle.com/code/cast42/simple-svd-movie-recommender

In [ ]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(movies_X_users.to_numpy(), k = 50) #o que são essas k features

print(f"Matriz original{movies_X_users.shape} decomposta em U{U.shape}, sigma {sigma.shape} e Vt{Vt.shape}.")

Matriz original(13322, 27321) decomposta em U(13322, 50), sigma (50,) e Vt(50, 27321).


In [ ]:
sigma_diag_matrix=np.diag(sigma) #sigma é um array contendo a diagonal
all_user_predicted_ratings = np.dot(np.dot(U, sigma_diag_matrix), Vt)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = movies_X_users.columns, index=movies_X_users.index)
preds_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,208531,208545,208683,208715,208737,208787,208793,208795,208939,209163
userId,,,,,,,,,,,,,,,,,,,,,
1,0.772561,-0.031004,-0.045287,-0.036903,0.009587,-0.207948,0.064104,-0.032135,-0.001484,0.123840,...,-0.000974,0.000111,0.002047,0.000240,-0.001915,-0.001339,-0.002163,0.002527,-0.001393,0.006462
2,4.280314,0.440407,0.044153,-0.013698,-0.120013,0.462136,0.194920,0.093330,-0.101376,0.448111,...,-0.002205,-0.001829,-0.011183,0.002208,-0.003763,-0.002593,0.001678,0.016950,-0.006913,0.006643
3,1.260915,0.471058,-0.260996,-0.133090,-0.017546,0.523084,0.170512,-0.146719,-0.173209,-0.390248,...,0.002390,0.007356,0.016333,-0.002372,0.020703,0.015702,-0.006491,-0.020402,0.011331,-0.002112
4,2.631603,0.220179,-0.178007,-0.029201,-0.084359,0.069683,0.065234,-0.031115,-0.009492,0.328353,...,0.010204,-0.000380,0.007210,0.003881,0.017068,0.002440,-0.006303,0.001462,0.002747,0.007366
5,4.305501,0.937399,1.366239,0.108312,1.150614,1.609428,1.093901,0.065092,0.444644,1.228014,...,-0.002498,-0.000333,-0.001014,-0.001421,-0.006514,0.001098,0.001582,-0.000176,-0.002648,0.002661


In [ ]:
def get_high_recommended_movies(userId):
    movies_rated_by_user = movies_X_users.loc[userId]
    movies_high_rated_by_user =  movies_rated_by_user[movies_rated_by_user > 4.5].index
    movies_recommended_for_user = preds_df.loc[userId]
    movies_high_recommend_for_user = movies_recommended_for_user[movies_recommended_for_user > 4].index
    return set(movies_high_recommend_for_user) - set(movies_high_rated_by_user)

In [ ]:
user = 1

rec = get_high_recommended_movies(user)
rec_ = pd.DataFrame(index=list(rec), columns=['Título', 'Nota'])
rec_.index.name='movieId'
rec_['Título'] = listar_nomes_filmes(rec)
for id in rec:
    rec_.at[id,'Nota'] = preds_df.loc[user,id]
rec_



C:\Users\rodri\AppData\Local\Temp\ipykernel_17104\119331543.py:28: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  return movies.loc[indices]['title'].values.tolist()


,Título,Nota
movieId,,
